# Install detectron2

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install roboflow

import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow
import os
import shutil
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data.datasets import register_coco_instances

# Get dataset

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="jrkIR8tZdg5Y8uIaKmwZ")
project = rf.workspace("smart-fridge-jvm9v").project("smart-fridge-p5rw0")
version = project.version(9)
dataset = version.download("coco")

# Register Dataset

In [ ]:
register_coco_instances("my_dataset_train", {}, "/content/Smart-Fridge-9/train/_annotations.coco.json", "/content/Smart-Fridge-9/train")
register_coco_instances("my_dataset_val", {}, "/content/Smart-Fridge-9/valid/_annotations.coco.json", "/content/Smart-Fridge-9/valid")
register_coco_instances("my_dataset_test", {}, "/content/Smart-Fridge-9/test/_annotations.coco.json", "/content/Smart-Fridge-9/test")

In [ ]:
my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train")
dataset_dicts = DatasetCatalog.get("my_dataset_train")

for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=my_dataset_train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

# Training

In [ ]:
class MyTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)

cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.00025

cfg.SOLVER.MAX_ITER = 500
cfg.SOLVER.STEPS = []

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 200
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5

cfg.TEST.EVAL_PERIOD = 50

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.85
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_test", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "my_dataset_test")
inference_on_dataset(trainer.model, val_loader, evaluator)

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = ("my_dataset_test", )
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
predictor = DefaultPredictor(cfg)
test_metadata = MetadataCatalog.get("my_dataset_test")

In [ ]:
image_path = "/content/Smart-Fridge-9/test/Batch112_jpg.rf.d029f73ef022a7d16f9cf624ac84e8be.jpg"

im = cv2.imread(image_path)
if im is None:
    print(f"Failed to load the image from {image_path}")
else:
    outputs = predictor(im)

    v = Visualizer(
        im[:, :, ::-1],
        metadata=test_metadata,
        scale=0.8
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
src = '/content/output'
dst = '/content/gdrive/My Drive/Georgios_Karagiannis/Code/D-level2'

shutil.copytree(src, dst)

In [ ]:
f = open('config.yml', 'w')
f.write(cfg.dump())
f.close()

In [ ]:
file_path = '/content/gdrive/MyDrive/Georgios_Karagiannis/Code/config.yml'

with open(file_path, 'w') as f:
    f.write(cfg.dump())

print(f"Configuration saved to {file_path}")

# Prediction

In [ ]:
cfg = get_cfg()
cfg.merge_from_file("/content/gdrive/MyDrive/Georgios_Karagiannis/Code/config.yml")
cfg.MODEL.WEIGHTS = "/content/gdrive/MyDrive/Georgios_Karagiannis/Code/D-level2/model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
predictor = DefaultPredictor(cfg)
metadata = MetadataCatalog.get("my_dataset_train")

In [ ]:
image_path = "/content/Smart-Fridge-9/test/Batch133_jpg.rf.1910542a6746e9fdc17969c73896f69e.jpg"

im = cv2.imread(image_path)
if im is None:
    print(f"Failed to load the image from {image_path}")
else:
    outputs = predictor(im)

    v = Visualizer(
        im[:, :, ::-1],
        metadata=metadata,
        scale=0.8
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    cv2_imshow(out.get_image()[:, :, ::-1])